In [1]:
import numpy as np 
import pandas as pd 
#import os
from pyomo.environ import *
print('Libraries were improt was successfully!')


Libraries were improt was successfully!


In [2]:
def read_data(DataFile):
    xlsLoad = pd.ExcelFile(DataFile)
    Bus = pd.read_excel(xlsLoad, 'bus').set_index('id')
    Gen = pd.read_excel(xlsLoad, 'gen').set_index('id')
    Line = pd.read_excel(xlsLoad, 'line').set_index('id')
    #id should be a column with unique information. 
    
    return Bus, Gen, Line

In [3]:
def DCOPF(Bus, Gen, Line):
    # define a pyomo model
    model = ConcreteModel()
    N=Bus.index
    G=Gen.index
    K=Line.index
    
    # define model variables and their limits
    model.p = Var(G, bounds=(0, None))
    model.f = Var(K)
    model.theta=Var(N)
    
    # Define the objective function
    model.obj = Objective(expr=sum(model.p[g]*Gen.loc[g, 'vCost1'] for g in G))
    
    # define model constraints
    model.gen_max = ConstraintList()
    model.flow = ConstraintList()
    model.flow_max = ConstraintList()
    model.node_balance = ConstraintList()
    model.slack=ConstraintList()
    
    # Add the constraints to the model
      
    # Generation maximum Limits:
    for g in G:
        model.gen_max.add(model.p[g]<=Gen.loc[g,'CapHigh'])
    
    # Power flow equations and limits
    for k in K:
        model.flow.add(model.f[k]==Line.loc[k,'B_dc (MW)']*
                       (model.theta[Line.loc[k,'from']]-model.theta[Line.loc[k,'to']]))
        model.flow_max.add(model.f[k]<=Line.loc[k,'CAPA'])
        model.flow_max.add(model.f[k]>=-Line.loc[k,'CAPA'])
    
    # Slack bus constraint
    model.slack.add(model.theta[1]==0)
    
    # Node balance constraints
    
    for n in N:
        demand = Bus.loc[n,'P-load']
        inflow = 0
        outflow = 0
        localgen = 0
        for k in K:
            if Line.loc[k,'to']==n:
                inflow += model.f[k]
            if Line.loc[k,'from']==n:
                outflow += model.f[k]
        for g in G:
            if Gen.loc[g,'node']==n:
                localgen += model.p[g]
        model.node_balance.add(localgen+inflow-outflow==demand)
    return model
    
                




In [4]:
# Read Data
#print(os.getcwd())
Bus, Gen, Line = read_data('RTS_Data.xlsx')
print("Data was read successfully.")
N=Bus.index
G=Gen.index
K=Line.index

# Construct Model
model = DCOPF(Bus, Gen, Line)
print("Construction of model complete.")

# Solving steps

# Identify the solver
opt = SolverFactory('glpk')

# Tell the model that you need the dual values too!
model.dual = Suffix(direction=Suffix.IMPORT)

# Solve the problem
Solution=opt.solve(model)

# Report the summary of solve
Solution.write(num=1)

# Report the output
for g in G:
    print('Generator ',g,'Output (MW):', model.p[g].value)
    
for k in K:
    print('Line ',k,'Flow (MW):', model.f[k].value)

for n in N:
    print('LMP for node ', n, 'in ($/MWh) is: ', model.dual[model.node_balance[n]])
    
for k in K:
    print('Flowgate price for line', k, 'in ($/MWh) in both directions are: '
          , model.dual[model.flow_max[2*k-1]],' and ', model.dual[model.flow_max[2*k]])


Data was read successfully.
Construction of model complete.
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 57521.9002783913
  Upper bound: 57521.9002783913
  Number of objectives: 1
  Number of constraints: 173
  Number of variables: 96
  Number of nonzeros: 334
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.015889883041381836
# -----------------------------------